In [87]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib import RDF, RDFS, OWL, XSD
import sys
sys.path.append('../')
sys.path.append('../andre')
import andre.utils as utils
import collections
SCHEMA = Namespace("http://schema.org/")
PBS = Namespace("http://www.example.org/pbs#")

In [94]:
graphe_Auteurs = Graph()
graphe_Auteurs.bind("schema","http://schema.org/", override=True, replace=True)
graphe_Auteurs.parse("../final_datasets/btlf_books/Graphes/grapheAuteurs_andre.ttl")

<Graph identifier=Na66522e13eaf4845aa70d36b47edbdcf (<class 'rdflib.graph.Graph'>)>

In [95]:
def extract_author_turtle(graph, author):
    last_name = str(graph.value(author, SCHEMA.familyName))
    first_name = str(graph.value(author, SCHEMA.givenName)) if graph.value(author, SCHEMA.givenName) else ""
    description = str(graph.value(author, SCHEMA.description)) if graph.value(author, SCHEMA.description) else ""
    return last_name, first_name, description

btlf_authors = {}

for author in graphe_Auteurs.subjects(RDF.type, utils.schema.Person):
    last_name, first_name, description = extract_author_turtle(graphe_Auteurs, author)
    # btlf_authors[first_name + " " + last_name] = author

    if last_name and first_name:
        btlf_authors[first_name + " " + last_name] = author
    elif last_name and not first_name:
        btlf_authors[last_name] = author
    elif not last_name and first_name:
        print("no last name", author)
    elif not last_name and not first_name:
        print("no first name", author)
    
    # print(last_name, first_name,  description)

print(btlf_authors)
len(btlf_authors)

{'Anthony Kemp': rdflib.term.URIRef('http://schema.org/author1'), 'Michel Honaker': rdflib.term.URIRef('http://schema.org/author10'), 'Dominique Piat': rdflib.term.URIRef('http://schema.org/author100'), 'Jules Supervielle': rdflib.term.URIRef('http://schema.org/author1000'), 'John Tiffany': rdflib.term.URIRef('http://schema.org/author10000'), 'Guilhem Flouzat': rdflib.term.URIRef('http://schema.org/author10001'), 'Cécile Térouanne': rdflib.term.URIRef('http://schema.org/author10002'), 'Rachel Williams': rdflib.term.URIRef('http://schema.org/author10003'), 'Lucy Letherland': rdflib.term.URIRef('http://schema.org/author10004'), 'Jean-Claude Dusaussoy': rdflib.term.URIRef('http://schema.org/author10005'), 'Yolande Moreau': rdflib.term.URIRef('http://schema.org/author10006'), 'Katherine J. Willis': rdflib.term.URIRef('http://schema.org/author10007'), 'Jean-Christophe Fournier': rdflib.term.URIRef('http://schema.org/author10008'), 'Aurex Verdon': rdflib.term.URIRef('http://schema.org/author

18144

In [103]:
g = Graph()
g.bind("schema","http://schema.org/", override=True, replace=True)
g.bind("pbs","http://www.example.org/pbs#", override=True, replace=True)

g.parse("../final_datasets/constellations.ttl", format="turtle")
len(list(g.subjects(RDF.type, utils.schema.Book)))

11267

In [104]:
array_size_counter = collections.defaultdict(lambda: 0)
array_size_names = collections.defaultdict(lambda: [])
author_match_counter = 0
id = 0
for book in g.subjects(RDF.type, utils.schema.Book):
    book_data = utils.extract_data_constellation(g, book)
    author_name = book_data.book_author
    if author_name in btlf_authors.keys():
        author_match_counter +=1
        g.add((book, PBS.authorURI, btlf_authors[author_name])) # on ajoute les nouveaux auteurs sans séparer les nom, prenom
    else:
        author_uri = SCHEMA['authorConstellations' + str(id)]
        graphe_Auteurs.add((author_uri, RDF.type, SCHEMA['Person']))
        graphe_Auteurs.add((author_uri, SCHEMA.familyName,Literal(author_name)))
        g.add((book, PBS.authorURI, author_uri)) # on ajoute les nouveaux auteurs sans séparer les nom, prenom

    id += 1
    if " et " not in author_name and " and " not in author_name and "d'après"  not in author_name and "d’après" not in author_name and "&" not in author_name and "with" not in author_name: # multiple authors need to be fixed
        name_array = author_name.rsplit(" ")
        if len(name_array) == 0:
            array_size_counter["0"] += 1
            array_size_names["0"].append(name_array)
        elif len(name_array) == 1:
            array_size_counter["1"] += 1
            array_size_names["1"].append(name_array)
        elif len(name_array) == 2:
            array_size_counter["2"] += 1
            array_size_names["2"].append(name_array)
        elif len(name_array) == 3:
            array_size_counter["3"] += 1
            array_size_names["3"].append(name_array)
        elif len(name_array) == 4:
            array_size_counter["4"] += 1
            array_size_names["4"].append(name_array)
        elif len(name_array) == 5:
            array_size_counter["5"] += 1
            array_size_names["5"].append(name_array)
        elif len(name_array) == 6:
            array_size_counter["6"] += 1
            array_size_names["6"].append(name_array)
        elif len(name_array) == 7:
            array_size_counter["7"] += 1
            array_size_names["7"].append(name_array)
        elif len(name_array) == 8:
            array_size_counter["8"] += 1
        elif len(name_array) == 9:
            array_size_counter["9"] += 1
author_match_counter

7680

In [102]:
with open("../final_datasets/btlf_books/Graphes/grapheAuteurs_BTLF_Constellations.ttl","wb") as fichier:
    graphe_Auteurs.serialize(fichier) 

with open("../final_datasets/constellations_auteursURI.ttl","wb") as fichier:
    g.serialize(fichier) 

In [62]:
array_size_counter

defaultdict(<function __main__.<lambda>()>,
            {'2': 9147, '1': 457, '3': 951, '4': 99, '6': 1, '5': 2, '7': 1})

In [67]:
array_size_names["3"]

[['Evan', 'H.', 'Rhodes'],
 ['Randall', 'de', 'Sève'],
 ['Virginie', 'Jouannet', 'Roussel'],
 ['Alain', 'M.', 'Bergeron'],
 ['Olivier', 'de', 'Solminihac'],
 ['Robie', 'H.', 'Harris'],
 ['Agnès', 'de', 'Lestrade'],
 ['Clive', 'A.', 'Lawton'],
 ['Michel', 'Van', 'Zeveren'],
 ['Élisabeth', 'de', 'Lambilly'],
 ['Guido', 'van', 'Genechten'],
 ['Nicholas', 'John', 'Frith'],
 ['Christine', 'Sioui', 'Wawanoloath'],
 ['Clement', 'Clarke', 'Moore'],
 ['Margaret', 'Wise', 'Brown'],
 ['Peter', 'H.', 'Reynolds'],
 ['Sandra', 'Le', 'Guen'],
 ['Magali', 'Le', 'Huche'],
 ['Chris', 'Van', 'Allsburg'],
 ['Frank', 'Cottrell', 'Boyce'],
 ['Judith', 'Janda', 'Presnall'],
 ['J.', 'Patrick', 'Lewis'],
 ['Wendelin', 'Van', 'Draanen'],
 ['Angela', 'May', 'George'],
 ['Jon', 'J.', 'Muth'],
 ['Alain', 'M.', 'Bergeron'],
 ['Alain', 'M.', 'Bergeron'],
 ['David', 'A.', 'Robertson'],
 ['Alain', 'M.', 'Bergeron'],
 ['Jiang', 'Hong', 'Chen'],
 ['Paul', 'B.', 'Janeczko'],
 ['Lucy', 'Ruth', 'Cummins'],
 ['Kimberly', 'B

In [66]:
array_size_names["4"]

[['Philippe', 'de', 'La', 'Cotardière'],
 ['Jean', 'de', 'La', 'Fontaine'],
 ['José', 'Mauro', 'de', 'Vasconcelos'],
 ['Camilla', 'de', 'la', 'Bédoyère'],
 ['Camilla', 'de', 'la', 'Bédoyère'],
 ['Jean', 'de', 'La', 'Fontaine'],
 ['Jeanne-Marie', 'Leprince', 'de', 'Beaumont'],
 ['Ernst', 'Theodor', 'Amadeus', 'Hoffmann'],
 ['Martha', 'E.', 'H.', 'Rustad'],
 ['Jr.', 'Martin', 'Luther', 'King'],
 ['Guy', 'de', 'la', 'Bédoyère'],
 ['John', 'Ronald', 'Reuel', 'Tolkien'],
 ['Jean', 'de', 'La', 'Fontaine'],
 ['Michel', 'Da', 'Costa', 'Gonçalves'],
 ['Anne-Laure', 'Fournier', 'Le', 'Ray'],
 ['Camilla', 'de', 'la', 'Bédoyère'],
 ['Dominique', 'de', 'Saint', 'Mars'],
 ['Melissa', 'de', 'la', 'Cruz'],
 ['Ursula', 'K.', 'Le', 'Guin'],
 ['Matt', 'de', 'la', 'Peña'],
 ['Catherine', 'Wihtol', 'de', 'Wenden'],
 ['Camilla', 'de', 'la', 'Bédoyère'],
 ['Robert', 'D.', 'San', 'Souci'],
 ['Camilla', 'de', 'la', 'Bédoyère'],
 ['Jean', 'de', 'La', 'Fontaine'],
 ['Ursula', 'K.', 'Le', 'Guin'],
 ['John', 'Rona

# vu que c'est compliqué de séparer le nom et le prénom
    -> tout mettre en mode prenom + nom
    -> refaire constellation pour avoir nom, prenom